In [26]:
import uxarray as ux
import xarray as xr
import numpy as np

### Exploring how MPAS-Ocean grids and datasets are represented in `uxarray`

See https://uxarray.readthedocs.io/en/v2024.01.1/examples/004-working-with-mpas-grids.html

In [52]:
mesh_path = '/global/cfs/projectdirs/e3sm/inputdata/ocn/mpas-o/EC30to60E2r2/ocean.EC30to60E2r2.210210.nc'

bmm_filepath = '/pscratch/sd/b/bmoorema/run_001_062/'
dataset_path = bmm_filepath + '20210421_sim7_CORE_60to30E2r2.mpaso.hist.am.timeSeriesStatsMonthly.0063-12-01.nc'
dso = ux.open_dataset(
    mesh_path,
    dataset_path,
    decode_timedelta=True
)

#### Add integer index coordinates for each dimension

This is only required for multi-dimensional `.sel` methods later!

In [53]:
# Add grid coordinates
dso = dso.assign_coords({
    c:xr.DataArray(np.arange(dso.sizes[c]), dims=(c,))
    for c in dso.dims
})
dso = dso.assign_coords({"two":xr.DataArray([0,1], dims=("two",))})

### Mask faces on edges

In [63]:
faces_on_edges = dso.uxgrid.edge_face_connectivity.assign_coords({"n_edge":dso.n_edge, "two":dso.two})
valid_face_mask = faces_on_edges >= 0
faces_on_edges = faces_on_edges.where(valid_face_mask, 0) # arbitrarily set all implicit land faces to cell 0

In [64]:
dso["timeMonthly_avg_density_onEdge"] = (
    dso.timeMonthly_avg_density
    .chunk({"nVertLevels":1}) # make chunks smaller to keep memory impact low
    .sel(n_face=faces_on_edges) # select faces
    .where(valid_face_mask, np.nan) # remember to mask the implicit land faces!
    .mean("two") # take a mean across densities
)